![bitstring](https://raw.githubusercontent.com/scott-griffiths/bitstring/main/doc/bitstring_logo_small.png)

# A Walkthrough of the bitstring Module

## Introduction

The aim of the `bitstring` module is make dealing with binary data in Python as easy as possible.
In this notebook we'll go through some features of the module to help you get started using it.

Only a few of the module's features will be covered in this walkthrough; see the [documentation](https://bitstring.readthedocs.io/en/stable/index.html) for a more thorough guide.
The whole of this notebook can be safely skipped if you prefer to start with the manual.

## Getting started

First we need to install the module, which is just a call to `pip install bitstring`.

In [ ]:
%pip install bitstring
import bitstring
from bitstring import BitArray
import math


We've imported the `BitArray` class directly to save some typing.
This is just a mutable container for binary data. Later on we'll also use the `BitStream` class which adds a bit position and reading methods to treat the data as a stream.
There are also immutable versions of both these classes that we won't be using here.

We can now create a couple of bitstrings:

In [2]:
a = BitArray('0xff01')
b = BitArray('0b110')

The first of these we made from the hexadecimal string `0xff01` - the `0x` prefix makes it hexadecimal just as `0b` means binary and `0o` means octal.
Each hex digit represents four bits, so we have a bitstring of length 16 bits.

The second was created from a binary string.
In this case it is just three bits long.
Don't worry about it not being a whole number of bytes long, that's all been taken care of internally.

<div class="alert alert-block alert-info">
<b>Note:</b>

Be sure to remember the quotes around the hex and binary strings.
If you forget them you would just have an ordinary Python integer, which would instead create a bitstring of that many '0' bits.
For example `0xff01` is the same as the base-10 number 65281, so `BitArray(0xff01)` would consist of 65281 zero bits!
</div>

There are lots of things we can do with our new bitstrings, the simplest of which is just to print them:

In [3]:
print(a)
print(b)

0xff01
0b110


Now you would be forgiven for thinking that the strings that we used to create the two bitstrings had just been stored to be given back when printed, but that's not the case.
Every bitstring should be considered just as a sequence of bits.
As we'll see there are lots of ways to create and manipulate them, but they have no memory of how they were created.
When they are printed they just pick the simplest hex or binary representation of themselves.
If you prefer you can pick the representation that you want:

In [4]:
print(a.bin)
print(b.oct)
print(b.int)
print(a.bytes)

1111111100000001
6
-2
b'\xff\x01'



There are a few things to note here:

* To get the different interpretations of the binary data we use properties such as `bin`, `hex`, `oct`, `int` and `bytes`. You can probably guess what these all mean, but you don't need to know quite yet. The properties are calculated when you ask for them rather than being stored as part of the object itself.
* Many of the interpretations have single letter aliases, and interpretations can also have bit lengths appended to them. This allows expressions such as `a.u32 = 900` which will set `a` to the 32 bit representation of the unsigned integer `900`. You're not restricted to the usual bit lengths, so something like `a.i5 = -8` will work as well.

Great - let's try some more:

In [5]:
try:
    b.hex
except bitstring.InterpretError as e:
    print(e)

Cannot convert to hex unambiguously - not a multiple of 4 bits long.



Oh dear - this throws a `bitstring.InterpretError` error.
The problem we have here is that `b` is 3 bits long, whereas each hex digit represents 4 bits.
This means that there is no unambiguous way to represent it in hexadecimal.
There are similar restrictions on other interpretations (octal must be a multiple of 3 bits, bytes a multiple of 8 bits etc.)

An exception is raised rather than trying to guess the best hex representation as there are a multitude of ways to convert to hex.
I occasionally get asked why it doesn't just do the 'obvious' conversion, which is invariably what that person expects from his own field of work.
This could be truncating bits at the start or end, or padding at the start or end with either zeros or ones.
Rather than try to guess what is meant we just raise an exception - if you want a particular behaviour then write it explicitly:


In [6]:
print((b + [0]).hex)
print(([0] + b).hex)

c
6



Here we've added a zero bit first to the end and then to the start.
Don't worry too much about how it all works, but just to give you a taster the zero bit `[0]` could also have been written as `BitArray([0])`, `BitArray('0b0')`, `BitArray(bin='0')`, `'0b0'` or just `1` (this final option isn't a typo, it means construct a bitstring of length one, with all the bits initialised to zero - it does look a bit confusing though which is why I prefer `[0]` and `[1]` to represent single bits).
Take a look at [the auto initialiser](https://bitstring.readthedocs.io/en/stable/creation.html#the-auto-initialiser) for more details.

## Modifying bitstrings

A `BitArray` can be treated just like a list of bits. You can slice it, delete sections, insert new bits and more using standard index notation:

In [7]:
print(a[3:9])
del a[-6:]
print(a)

0b111110
0b1111111100



The slicing works just as it does for other containers, so the deletion above removes the final six bits.

If you ask for a single item, rather than a slice, a boolean is returned. Naturally enough `1` bits are `True` whereas `0` bits are `False`.

In [8]:
print(a[0])
print(a[-1])

True
False



To join together bitstrings you can use a variety of methods, including `append`, `prepend`, `insert`, and plain `+` or `+=` operations::

In [9]:
a.prepend('0b01')
a.append('0o7')
a += '0x06'


Here we first put two bits at the start of `a`, then three bits on the end (a single octal digit) and finally another byte (two hex digits) on the end.

Note how we are just using ordinary strings to specify the new bitstrings we are adding.
These get converted automatically to the right sequence of bits.

<div class="alert alert-block alert-info">
<b>Note:</b>
 The length in bits of bitstrings specified with strings depends on the number of characters, including leading zeros.
 So each hex character is four bits, each octal character three bits and each binary character one bit.
 </div>

## Finding and replacing

A `find` is provided to search for bit patterns within a bitstring.
You can choose whether to search only on byte boundaries or at any bit position:

In [10]:
a = BitArray('0xa9f')
a.find('0x4f')

(3,)

Here we have found the `0x4f` byte in our bitstring, though it wasn't obvious from the hexadecimal as it was at bit position 3.
To see this clearer consider this equality:


In [11]:
a == '0b101, 0x4f, 0b1'

True

in which we've broken the bitstring into three parts to show the found byte.
This also illustrates using commas to join bitstring sections.

## Constructing a bitstring

Let's say you have a specification for a binary file type (or maybe a packet specification etc.) and you want to create a bitstring quickly and easily in Python.
For this example I'm going to use a header from the MPEG-2 video standard.
Here's how the header is described in the standard:

|sequence_header()                  | No. of bits  |  Mnemonic|
|-----------------------------------|--------------|----------|
|sequence_header_code               | 32           |  bslbf   |
|horizontal_size_value              | 12           |  uimsbf  |
|vertical_size_value                | 12           |  uimsbf  |
|aspect_ratio_information           | 4            |  uimsbf  |
|frame_rate_code                    | 4            |  uimsbf  |
|bit_rate_value                     | 18           |  uimsbf  |
|marker_bit                         | 1            |  bslbf   |
|vbv_buffer_size_value              | 10           |  uimsbf  |
|constrained_parameters_flag        | 1            |  bslbf   |
|load_intra_quantiser_matrix        | 1            |  uimsbf  |
|if (load_intra_quantiser_matrix)                             |
|{ intra_quantiser_matrix[64] }     | 8*64         |  uimsbf  |
|load_non_intra_quantiser_matrix    | 1            |  uimsbf  |
|if (load_non_intra_quantiser_matrix)                         |
|{ non_intra_quantiser_matrix[64] } | 8*64         |  uimsbf  |
|next_start_code()                  |              |          |

The mnemonics mean things like uimsbf = 'Unsigned integer, most significant bit first'.

So to create a sequence_header for your particular stream with width of 352 and height of 288 you could start like this:


In [12]:
s = BitArray()
s.append('0x000001b3')  # the sequence_header_code
s.append('uint12=352') # 12 bit unsigned integer
s.append('uint12=288')
print(s.hex)

000001b3160120


which is fine, but if you wanted to be a bit more concise you could just write

In [13]:
s = BitArray('0x000001b3, uint12=352, uint12=288')

This is better, but it might not be a good idea to have the width and height hard-wired in like that.
We can make it more flexible by using a format string and the `pack` function:

In [14]:
width, height = 352, 288
s = bitstring.pack('0x000001b3, 2*u12', width, height)

where we have also used `2*u12` as shorthand for `uint12, uint12`.

The [`pack`](https://bitstring.readthedocs.io/en/stable/functions.html#bitstring.pack) function can also take a dictionary as a parameter which can replace the tokens in the format string.
For example:

In [15]:
fmt = 'sequence_header_code, \
       uint12=horizontal_size_value, \
           uint12=vertical_size_value, \
           uint4=aspect_ratio_information, '
           # ...
d = {'sequence_header_code': '0x000001b3',
         'horizontal_size_value': 352,
         'vertical_size_value': 288,
         'aspect_ratio_information': 1,
         # ...
        }

s = bitstring.pack(fmt, **d)
repr(s)

"BitStream('0x000001b31601201')"

## Parsing bitstreams

You might have noticed that `pack` returned a `BitStream` rather than a `BitArray`.
This isn't a problem as the `BitStream` class just adds a few stream-like qualities to `BitArray` which we'll take a quick look at here.

The stream-ness of this object is via its bit position, and various reading and peeking methods.
First let's try a read or two, and see how this affects the bit position:


In [16]:
print(s.pos)
print(repr(s.read(24)))
print(s.pos)
print(s.read('hex8'))
print(s.pos)

0
BitStream('0x000001')
24
b3
32


First we read 24 bits, which returned a new `BitStream` object, then we used a format string to read 8 bits interpreted as a hexadecimal string.
We know that the next two sets of 12 bits were created from integers, so to read them back we can say

In [17]:
s.readlist('2*u12')

[352, 288]


If you don't want to use a bitstream then you can always use [`unpack`](https://bitstring.readthedocs.io/en/stable/bits.html#bitstring.Bits.unpack).
This takes much the same form as [`readlist`](https://bitstring.readthedocs.io/en/stable/constbitstream.html#bitstring.ConstBitStream.readlist) except it just unpacks from the start of the bitstring.
For example:

In [18]:
s.unpack('bytes4, 2*u12, uint4')

[b'\x00\x00\x01\xb3', 352, 288, 1]

# Worked Examples

Below are a few examples of using the bitstring module, as I always find that a good example can help more than a lengthy reference manual.

## Hamming distance

The Hamming distance between two bitstrings is the number of bit positions in which the two bitstrings differ.
So for example the distance between `0b00110` and `0b01100` is 2 as the second and fourth bits are different.

Let's write a function that calculates the Hamming weight of two bitstrings.


In [19]:
def hamming_weight(a, b):
    return (BitArray(a)^b).count(True)

hamming_weight('0b00110', '0b01100')

2

Er, that's it. The `^` is a bit-wise exclusive or, which means that the bits in `a^b` are only set if they differ in `a` and `b`.
The `count` method just counts the number of 1 (or True) bits.

## Sieve of Eratosthenes

The sieve of Eratosthenes is an ancient (and _very_ inefficient) method of finding prime numbers.
The algorithm starts with the number 2 (which is prime) and marks all of its multiples as not prime, it then continues with the next unmarked integer (which will also be prime) and marks all of its multiples as not prime.

[![Sieve animation](https://upload.wikimedia.org/wikipedia/commons/b/b9/Sieve_of_Eratosthenes_animation.gif)](https://commons.wikimedia.org/wiki/File:Sieve_of_Eratosthenes_animation.svg)

So to find all primes under a hundred million you could write:

In [20]:
# Create a BitArray with a hundred million 'one' bits
limit = 100_000_000
is_prime = BitArray(limit)
is_prime.set(True)
# Manually set 0 and 1 to be not prime.
is_prime.set(False, [0, 1])
# For every other integer, if it's set as prime then unset all of its multiples
for i in range(2, math.ceil(math.sqrt(limit))):
    if is_prime[i]:
        is_prime.set(False, range(i*i, limit, i))

print(f"There are {is_prime.count(True)} primes less than {limit},")
print(f"the largest one of which is {is_prime.rfind('0b1')[0]}")
print(f"and there are {len(list(is_prime.findall('0b101')))} twin primes.")

There are 5761455 primes less than 100000000,
the largest one of which is 99999989
and there are 440312 twin primes.


We find the largest prime with a reverse find ([`rfind`](https://bitstring.readthedocs.io/en/stable/bits.html#bitstring.Bits.rfind)) looking for a single set bit.
For twin primes (primes which differ by 2) we use [`findall`](https://bitstring.readthedocs.io/en/stable/bits.html#bitstring.Bits.findall) to look for the binary sequence `101` which returns a generator for the bit positions.

To see the pattern of the primes we could use the [pretty print](https://bitstring.readthedocs.io/en/stable/bits.html#bitstring.Bits.pp) method:

In [21]:
is_prime[0:1000].pp('bin, hex', width=88)

   0: 00110101 00010100 01010001 00000101 00000100 01010001   35 14 51 05 04 51
  48: 00000100 00010100 00010001 01000001 00010000 01000000   04 14 11 41 10 40
  96: 01000101 00010100 01000000 00000001 00010000 01010000   45 14 40 01 10 50
 144: 00000101 00000100 00010001 00000100 00010100 00000001   05 04 11 04 14 01
 192: 01000101 00000000 00010000 00000001 00010100 01000001   45 00 10 01 14 41
 240: 01000000 00010000 01000001 00000101 00000100 01010000   40 10 41 05 04 50
 288: 00000100 00000000 00010001 01000100 00000000 00010000   04 00 11 44 00 10
 336: 01000000 00010100 01000001 00000001 00000100 00010001   40 14 41 01 04 11
 384: 00000100 00000100 01000000 01000000 00010100 00000001   04 04 40 40 14 01
 432: 01000001 00010000 01000000 01000101 00010000 00000001   41 10 40 45 10 01
 480: 00000001 00010000 00010001 00000100 00000000 01010000   01 10 11 04 00 50
 528: 00000000 00000100 00010000 00000100 00010000 01010000   00 04 10 04 10 50
 576: 01000000 00010000 01000001 0100000

I'll leave optimising the algorithm as an exercise for the reader, but it illustrates both bit checking and setting.
One reason you might want to use a bitstring for this purpose (instead of a plain list for example) is that the million bits only take up a million bits in memory, whereas for a list of integers it would be much more.

## Next Steps

There is fairly extensive [documentation](https://bitstring.readthedocs.io/en/stable/) available.
The [Quick Reference](https://bitstring.readthedocs.io/en/stable/quick_reference.html) is a good place to quickly see what's available.

For any discussions / bug reports / feature requests see the homepage on [GitHub](https://github.com/scott-griffiths/bitstring/).